# QuartumSE Benchmark Suite

This notebook demonstrates the **unified benchmark suite** API for running publication-grade benchmarks.

## The 8 Tasks (Measurements Bible)

| Task | Name | Question |
|------|------|----------|
| 1 | Worst-Case Guarantee | What N* achieves max SE ≤ ε for all observables? |
| 2 | Average Target | What N* achieves mean SE ≤ ε? |
| 3 | Fixed Budget | What is the SE distribution at fixed N? |
| 4 | Dominance | Which protocol wins on more observables? |
| 5 | Pilot Selection | How much budget for pilot vs main run? |
| 6 | Bias-Variance | How does MSE decompose into bias² + variance? |
| 7 | Noise Sensitivity | How does performance degrade with noise? |
| 8 | Adaptive Efficiency | Can we reallocate budget based on pilot? |

## Benchmark Modes

| Mode | What it runs | Use case |
|------|--------------|----------|
| `basic` | Tasks 1, 3, 6 + basic report | Quick sanity check |
| `complete` | All 8 tasks + complete report | Publication benchmark |
| `analysis` | Complete + enhanced analysis | Deep dive with statistics |

All results are saved with **unique timestamped directories** - no overwrites!

In [1]:
# --- Setup ---
import sys
sys.path.insert(0, '../src')

from qiskit import QuantumCircuit

from quartumse import (
    run_benchmark_suite,
    BenchmarkMode,
    BenchmarkSuiteConfig,
    generate_observable_set,
    Observable,
    ObservableSet,
)

print("Setup complete!")

Setup complete!


---

## 1. Define Circuit and Observables

You can use **any Qiskit circuit** and **any set of observables**.

In [2]:
# --- Circuit ---
N_QUBITS = 4

def build_ghz(n_qubits: int) -> QuantumCircuit:
    """Build GHZ state preparation circuit."""
    qc = QuantumCircuit(n_qubits)
    qc.h(0)
    for i in range(1, n_qubits):
        qc.cx(i - 1, i)
    return qc

circuit = build_ghz(N_QUBITS)
print(circuit.draw('text'))

     ┌───┐               
q_0: ┤ H ├──■────────────
     └───┘┌─┴─┐          
q_1: ─────┤ X ├──■───────
          └───┘┌─┴─┐     
q_2: ──────────┤ X ├──■──
               └───┘┌─┴─┐
q_3: ───────────────┤ X ├
                    └───┘


In [3]:
# --- Observables ---
# Generate random observables with mixed localities
observables = []
for k in range(1, N_QUBITS + 1):
    obs_set = generate_observable_set(
        generator_id='random_pauli',
        n_qubits=N_QUBITS,
        n_observables=5,
        seed=42 + k,
        weight_distribution='fixed',
        fixed_weight=k,
    )
    observables.extend(list(obs_set.observables))

# Add GHZ stabilizers
observables.extend([
    Observable('Z' * N_QUBITS),
    Observable('X' * N_QUBITS),
])

obs_set = ObservableSet(
    observables=observables,
    observable_set_id='benchmark_observables',
    generator_id='mixed',
    generator_seed=42,
)

# Build locality map for analysis mode
locality_map = {}
for obs in observables:
    locality = sum(1 for c in obs.pauli_string if c != 'I')
    locality_map[obs.observable_id] = locality

print(f"Generated {len(obs_set)} observables")
from collections import Counter
loc_counts = Counter(locality_map.values())
for k in sorted(loc_counts.keys()):
    print(f"  K={k}: {loc_counts[k]} observables")

Generated 22 observables
  K=1: 5 observables
  K=2: 5 observables
  K=3: 5 observables
  K=4: 7 observables


---

## 2. Basic Benchmark

Quick sanity check with Tasks 1, 3, 6.

In [4]:
%%time
# --- Basic Benchmark ---
config = BenchmarkSuiteConfig(
    mode=BenchmarkMode.BASIC,
    n_shots_grid=[100, 500, 1000],
    n_replicates=5,  # Fewer for quick test
    seed=42,
    output_base_dir="benchmark_results",
)

result = run_benchmark_suite(
    circuit=circuit,
    observable_set=obs_set,
    circuit_id="ghz_4q",
    config=config,
)

BENCHMARK SUITE: BASIC
Run ID: ghz_4q_20260116_123551_37d0b281
Output: benchmark_results\ghz_4q_20260116_123551_37d0b281
Mode: basic

Step 1: Running base benchmark...
  Completed: 990 rows

Step 4: Generating reports...
  Basic report: benchmark_results\ghz_4q_20260116_123551_37d0b281\basic_report.md

BENCHMARK COMPLETE
Output directory: benchmark_results\ghz_4q_20260116_123551_37d0b281
Reports generated: ['basic', 'config', 'manifest']

CPU times: total: 4min 30s
Wall time: 1min 20s


In [5]:
# --- View Basic Results ---
print(f"Run ID: {result.run_id}")
print(f"Output: {result.output_dir}")
print(f"Reports: {list(result.reports.keys())}")
print()
print("Protocol summaries:")
for protocol, stats in result.summary.get('protocol_summaries', {}).items():
    print(f"  {protocol}: mean_se={stats.get('mean_se', 0):.4f}")

Run ID: ghz_4q_20260116_123551_37d0b281
Output: benchmark_results\ghz_4q_20260116_123551_37d0b281
Reports: ['basic', 'config', 'manifest']

Protocol summaries:
  direct_grouped: mean_se=0.0912
  direct_optimized: mean_se=0.0836
  classical_shadows_v0: mean_se=0.1588


---

## 3. Complete Benchmark (All 8 Tasks)

Publication-grade benchmark with all tasks.

In [6]:
%%time
# --- Complete Benchmark ---
config = BenchmarkSuiteConfig(
    mode=BenchmarkMode.COMPLETE,
    n_shots_grid=[100, 500, 1000, 5000],
    n_replicates=10,
    seed=42,
    epsilon=0.01,
    delta=0.05,
    output_base_dir="benchmark_results",
)

result = run_benchmark_suite(
    circuit=circuit,
    observable_set=obs_set,
    circuit_id="ghz_4q",
    config=config,
)

BENCHMARK SUITE: COMPLETE
Run ID: ghz_4q_20260116_124453_522a4ff6
Output: benchmark_results\ghz_4q_20260116_124453_522a4ff6
Mode: complete

Step 1: Running base benchmark...
  Completed: 2640 rows

Step 2: Running all 8 tasks...
  Completed: 12 task evaluations

Step 4: Generating reports...
  Basic report: benchmark_results\ghz_4q_20260116_124453_522a4ff6\basic_report.md
  Complete report: benchmark_results\ghz_4q_20260116_124453_522a4ff6\complete_report.md

BENCHMARK COMPLETE
Output directory: benchmark_results\ghz_4q_20260116_124453_522a4ff6
Reports generated: ['basic', 'complete', 'config', 'manifest']

CPU times: total: 12min 39s
Wall time: 3min 59s


In [7]:
# --- View Complete Results ---
print(f"Run ID: {result.run_id}")
print(f"All tasks completed: {len(result.all_task_results or {})}")
print()
if result.all_task_results:
    for task_id in sorted(result.all_task_results.keys()):
        print(f"  {task_id}")

Run ID: ghz_4q_20260116_124453_522a4ff6
All tasks completed: 12

  task1_classical_shadows_v0
  task1_direct_grouped
  task1_direct_optimized
  task2_classical_shadows_v0
  task2_direct_grouped
  task2_direct_optimized
  task3_classical_shadows_v0
  task3_direct_grouped
  task3_direct_optimized
  task6_classical_shadows_v0
  task6_direct_grouped
  task6_direct_optimized


---

## 4. Full Analysis Mode

Complete benchmark + enhanced analysis:
- N* interpolation (power-law fitting)
- Per-observable crossover
- Locality correlation
- Bootstrap hypothesis testing
- Cost-normalized metrics
- Multi-pilot fraction analysis

In [8]:
%%time
# --- Full Analysis ---
config = BenchmarkSuiteConfig(
    mode=BenchmarkMode.ANALYSIS,
    n_shots_grid=[100, 500, 1000, 5000],
    n_replicates=20,
    seed=42,
    epsilon=0.01,
    delta=0.05,
    shadows_protocol_id="classical_shadows_v0",
    baseline_protocol_id="direct_grouped",
    output_base_dir="benchmark_results",
)

result = run_benchmark_suite(
    circuit=circuit,
    observable_set=obs_set,
    circuit_id="ghz_4q",
    config=config,
    locality_map=locality_map,
)

BENCHMARK SUITE: ANALYSIS
Run ID: ghz_4q_20260116_125630_8c40e800
Output: benchmark_results\ghz_4q_20260116_125630_8c40e800
Mode: analysis

Step 1: Running base benchmark...
  Completed: 5280 rows

Step 2: Running all 8 tasks...
  Completed: 12 task evaluations

Step 3: Running comprehensive analysis...
  Comprehensive analysis complete

Step 4: Generating reports...
  Basic report: benchmark_results\ghz_4q_20260116_125630_8c40e800\basic_report.md
  Complete report: benchmark_results\ghz_4q_20260116_125630_8c40e800\complete_report.md
  Analysis report: benchmark_results\ghz_4q_20260116_125630_8c40e800\analysis_report.md
  Analysis JSON: benchmark_results\ghz_4q_20260116_125630_8c40e800\analysis.json

BENCHMARK COMPLETE
Output directory: benchmark_results\ghz_4q_20260116_125630_8c40e800
Reports generated: ['basic', 'complete', 'analysis', 'analysis_json', 'config', 'manifest']

CPU times: total: 25min 51s
Wall time: 7min 40s


In [9]:
# --- Analysis Summary ---
print(f"Run ID: {result.run_id}")
print(f"Output: {result.output_dir}")
print()

if result.analysis:
    print("Analysis Summary:")
    for key, value in result.analysis.summary.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")

Run ID: ghz_4q_20260116_125630_8c40e800
Output: benchmark_results\ghz_4q_20260116_125630_8c40e800

Analysis Summary:
  n_protocols: 3
  n_observables: 22
  n_shots_evaluated: 4
  max_shots: 5000
  shadows_mean_se: 0.0725
  baseline_mean_se: 0.0405
  shadows_vs_baseline_ratio: 1.7883
  winner_at_max_n: direct_grouped
  shadows_wins_fraction: 0.4545
  baseline_wins_fraction: 0.5455
  optimal_pilot_fraction: 0.1000


In [10]:
# --- Display Reports ---
from IPython.display import display, Markdown

print("Generated Reports:")
for name, path in result.reports.items():
    print(f"  {name}: {path}")

# Display the analysis report
if 'analysis' in result.reports:
    report_content = result.reports['analysis'].read_text()
    display(Markdown(report_content))

Generated Reports:
  basic: benchmark_results\ghz_4q_20260116_125630_8c40e800\basic_report.md
  complete: benchmark_results\ghz_4q_20260116_125630_8c40e800\complete_report.md
  analysis: benchmark_results\ghz_4q_20260116_125630_8c40e800\analysis_report.md
  analysis_json: benchmark_results\ghz_4q_20260116_125630_8c40e800\analysis.json
  config: benchmark_results\ghz_4q_20260116_125630_8c40e800\config.json
  manifest: benchmark_results\ghz_4q_20260116_125630_8c40e800\manifest.json


# Comprehensive Benchmark Analysis

**Run ID:** ghz_4q_20260116_125630_8c40e800
**Protocols:** direct_grouped, classical_shadows_v0, direct_optimized
**Observables:** 22
**Shot Grid:** [100, 500, 1000, 5000]

---

## Executive Summary

- **n_protocols:** 3
- **n_observables:** 22
- **n_shots_evaluated:** 4
- **max_shots:** 5000
- **shadows_mean_se:** 0.07249165815911877
- **baseline_mean_se:** 0.04053672828641835
- **shadows_vs_baseline_ratio:** 1.788295731390014
- **winner_at_max_n:** direct_grouped
- **shadows_wins_fraction:** 0.45454545454545453
- **baseline_wins_fraction:** 0.5454545454545454
- **optimal_pilot_fraction:** 0.1

---

## Task Results

### Worst Case


**Enhanced Analysis:**

### Average Target


### Fixed Budget


### Dominance

- crossover_n: None
- always_a_better: False
- always_b_better: True
- metric_used: mean_se

### Pilot Selection

- pilot_n: 100
- target_n: 5000
- selection_accuracy: 0.55
- regret: 0.0038921702451488933
- criterion_type: truth_based

### Bias Variance


---

## Statistical Significance

| N | Diff. P-value | K-S P-value | Reject Null | SSF (95% CI) |
|---|---------------|-------------|-------------|--------------|
| 100 | 0.0000 | 0.0000 | Yes | 0.40 [0.34, 0.48] |
| 500 | 0.0000 | 0.0000 | Yes | 0.33 [0.29, 0.38] |
| 1000 | 0.0000 | 0.0000 | Yes | 0.32 [0.28, 0.36] |
| 5000 | 0.0000 | 0.0000 | Yes | 0.31 [0.27, 0.36] |

---

## Performance by Locality

---

## Per-Observable Crossover Analysis

- Protocol A (classical_shadows_v0) wins on 45.5% of observables
- Protocol B (direct_grouped) wins on 54.5% of observables
- Crossover exists for 0.0% of observables

---

## Multi-Pilot Fraction Analysis

| Pilot % | Accuracy | Mean Regret |
|---------|----------|-------------|
| 2% | 85.0% | 0.0005 |
| 5% | 85.0% | 0.0005 |
| 10% | 100.0% | 0.0000 |
| 20% | 100.0% | 0.0000 |

**Optimal pilot fraction:** 10%

---

## Interpolated N* (Power-Law)

- **direct_grouped:** N* = 82803 (RÂ² = 1.000)
- **direct_optimized:** N* = 63872 (RÂ² = 1.000)
- **classical_shadows_v0:** N* = 446135 (RÂ² = 1.000)


---

## 5. Custom Circuit Example

Use any circuit you want!

In [ ]:
# --- Custom Circuit Example ---
# Example: Random circuit
import numpy as np

def build_random_circuit(n_qubits: int, depth: int, seed: int = 42) -> QuantumCircuit:
    """Build a random circuit."""
    rng = np.random.default_rng(seed)
    qc = QuantumCircuit(n_qubits)
    
    for _ in range(depth):
        # Random single-qubit gates
        for q in range(n_qubits):
            gate = rng.choice(['h', 'x', 'y', 'z', 's', 't'])
            getattr(qc, gate)(q)
        
        # Random CNOTs
        for q in range(0, n_qubits - 1, 2):
            if rng.random() > 0.5:
                qc.cx(q, q + 1)
    
    return qc

custom_circuit = build_random_circuit(4, 3)
print(custom_circuit.draw('text'))

# You can now run:
# result = run_benchmark_suite(custom_circuit, obs_set, circuit_id="random_4q_d3", config=config)

---

## 6. Task Summary Report

Clear quantitative answers to each of the 8 benchmark questions.

In [ ]:
# --- TASK SUMMARY REPORT ---
# Generate clear quantitative answers for each of the 8 tasks

import numpy as np
from collections import defaultdict

def generate_task_summary(result):
    """Generate a clear summary answering each task question."""
    
    print("=" * 80)
    print("BENCHMARK TASK SUMMARY REPORT")
    print("=" * 80)
    print(f"\nCircuit: {result.run_id.split('_')[0]}")
    print(f"Observables: {result.summary.get('n_observables', 'N/A')}")
    print(f"Protocols: {', '.join(result.summary.get('protocols', []))}")
    print(f"Shot Grid: {result.summary.get('n_shots_grid', [])}")
    print()
    
    long_form = result.long_form_results
    truth_values = result.ground_truth.truth_values if result.ground_truth else {}
    max_n = max(result.summary.get('n_shots_grid', [5000]))
    epsilon = 0.01  # Target precision
    
    # Group data by protocol and N
    by_protocol_n = defaultdict(lambda: defaultdict(list))
    for row in long_form:
        by_protocol_n[row.protocol_id][row.N_total].append(row)
    
    protocols = list(by_protocol_n.keys())
    
    # ========== TASK 1: Worst-Case Guarantee ==========
    print("-" * 80)
    print("TASK 1: WORST-CASE GUARANTEE")
    print("Question: What N* achieves max SE ≤ ε for ALL observables?")
    print("-" * 80)
    print(f"Target ε = {epsilon}")
    print()
    
    for protocol in protocols:
        n_star = None
        for n in sorted(by_protocol_n[protocol].keys()):
            rows = by_protocol_n[protocol][n]
            max_se = max(r.se for r in rows if r.se is not None)
            if max_se <= epsilon:
                n_star = n
                break
        
        if n_star:
            print(f"  {protocol}: N* = {n_star} shots")
        else:
            # Get max SE at largest N
            rows = by_protocol_n[protocol][max_n]
            max_se = max(r.se for r in rows if r.se is not None)
            print(f"  {protocol}: N* > {max_n} (max SE = {max_se:.4f} at N={max_n})")
    print()
    
    # ========== TASK 2: Average Target ==========
    print("-" * 80)
    print("TASK 2: AVERAGE TARGET")
    print("Question: What N* achieves mean SE ≤ ε?")
    print("-" * 80)
    print(f"Target ε = {epsilon}")
    print()
    
    for protocol in protocols:
        n_star = None
        for n in sorted(by_protocol_n[protocol].keys()):
            rows = by_protocol_n[protocol][n]
            mean_se = np.mean([r.se for r in rows if r.se is not None])
            if mean_se <= epsilon:
                n_star = n
                break
        
        if n_star:
            print(f"  {protocol}: N* = {n_star} shots")
        else:
            rows = by_protocol_n[protocol][max_n]
            mean_se = np.mean([r.se for r in rows if r.se is not None])
            print(f"  {protocol}: N* > {max_n} (mean SE = {mean_se:.4f} at N={max_n})")
    print()
    
    # ========== TASK 3: Fixed Budget Distribution ==========
    print("-" * 80)
    print("TASK 3: FIXED BUDGET DISTRIBUTION")
    print(f"Question: What is the SE distribution at N = {max_n}?")
    print("-" * 80)
    print()
    print(f"{'Protocol':<25} {'Mean SE':>10} {'Median SE':>10} {'Max SE':>10} {'Std SE':>10}")
    print("-" * 70)
    
    for protocol in protocols:
        rows = by_protocol_n[protocol][max_n]
        ses = [r.se for r in rows if r.se is not None]
        if ses:
            print(f"{protocol:<25} {np.mean(ses):>10.4f} {np.median(ses):>10.4f} "
                  f"{np.max(ses):>10.4f} {np.std(ses):>10.4f}")
    print()
    
    # ========== TASK 4: Dominance ==========
    print("-" * 80)
    print("TASK 4: DOMINANCE")
    print("Question: Which protocol wins on more observables?")
    print("-" * 80)
    print()
    
    # Compare at max N
    obs_winners = defaultdict(lambda: defaultdict(float))
    for protocol in protocols:
        rows = by_protocol_n[protocol][max_n]
        for row in rows:
            obs_winners[row.observable_id][protocol] = row.se
    
    wins = defaultdict(int)
    for obs_id, protocol_ses in obs_winners.items():
        if protocol_ses:
            winner = min(protocol_ses, key=protocol_ses.get)
            wins[winner] += 1
    
    total_obs = len(obs_winners)
    print(f"At N = {max_n}:")
    for protocol in protocols:
        win_count = wins[protocol]
        win_pct = 100 * win_count / total_obs if total_obs > 0 else 0
        print(f"  {protocol}: wins on {win_count}/{total_obs} observables ({win_pct:.1f}%)")
    
    if wins:
        overall_winner = max(wins, key=wins.get)
        print(f"\n  WINNER: {overall_winner}")
    print()
    
    # ========== TASK 5: Pilot Selection ==========
    print("-" * 80)
    print("TASK 5: PILOT SELECTION")
    print("Question: What fraction of budget should be used for pilot?")
    print("-" * 80)
    print()
    
    if result.analysis and hasattr(result.analysis, 'pilot_analysis') and result.analysis.pilot_analysis:
        pilot = result.analysis.pilot_analysis
        print(f"{'Pilot %':>10} {'Accuracy':>12} {'Mean Regret':>12}")
        print("-" * 40)
        for frac, res in sorted(pilot.results.items()):
            print(f"{frac*100:>9.0f}% {res.selection_accuracy*100:>11.1f}% {res.mean_regret:>12.4f}")
        print(f"\n  OPTIMAL PILOT: {pilot.optimal_fraction*100:.0f}% of budget")
    else:
        # Estimate based on data
        print("  (Requires analysis mode for detailed pilot study)")
        print("  Rule of thumb: 5-10% of budget for pilot")
    print()
    
    # ========== TASK 6: Bias-Variance Decomposition ==========
    print("-" * 80)
    print("TASK 6: BIAS-VARIANCE DECOMPOSITION")
    print("Question: How does MSE decompose into bias² + variance?")
    print("-" * 80)
    print()
    
    if truth_values:
        print(f"{'Protocol':<25} {'Bias²':>12} {'Variance':>12} {'MSE':>12}")
        print("-" * 65)
        
        for protocol in protocols:
            rows = by_protocol_n[protocol][max_n]
            
            # Group by observable
            by_obs = defaultdict(list)
            for row in rows:
                if row.observable_id in truth_values:
                    by_obs[row.observable_id].append(row.estimate)
            
            biases_sq = []
            variances = []
            for obs_id, estimates in by_obs.items():
                truth = truth_values[obs_id]
                mean_est = np.mean(estimates)
                bias = mean_est - truth
                var = np.var(estimates)
                biases_sq.append(bias**2)
                variances.append(var)
            
            if biases_sq:
                mean_bias_sq = np.mean(biases_sq)
                mean_var = np.mean(variances)
                mse = mean_bias_sq + mean_var
                print(f"{protocol:<25} {mean_bias_sq:>12.6f} {mean_var:>12.6f} {mse:>12.6f}")
    else:
        print("  (Requires ground truth for bias-variance decomposition)")
    print()
    
    # ========== TASK 7: Noise Sensitivity ==========
    print("-" * 80)
    print("TASK 7: NOISE SENSITIVITY")
    print("Question: How does performance degrade with noise?")
    print("-" * 80)
    print()
    print("  (Requires running benchmark with multiple noise profiles)")
    print("  Current run: ideal (noiseless) simulation")
    print()
    
    # ========== TASK 8: Adaptive Efficiency ==========
    print("-" * 80)
    print("TASK 8: ADAPTIVE EFFICIENCY")
    print("Question: Can budget reallocation improve results?")
    print("-" * 80)
    print()
    print("  (Requires two-stage adaptive protocol implementation)")
    print("  See pilot analysis (Task 5) for related insights")
    print()
    
    # ========== FINAL SUMMARY ==========
    print("=" * 80)
    print("EXECUTIVE SUMMARY")
    print("=" * 80)
    print()
    
    # Best protocol overall
    protocol_summaries = result.summary.get('protocol_summaries', {})
    if protocol_summaries:
        best_by_mean = min(protocol_summaries, key=lambda p: protocol_summaries[p].get('mean_se', float('inf')))
        best_by_max = min(protocol_summaries, key=lambda p: protocol_summaries[p].get('max_se', float('inf')))
        
        print(f"Best protocol (mean SE): {best_by_mean}")
        print(f"Best protocol (max SE):  {best_by_max}")
        print()
        
        # Shot savings factor
        if 'classical_shadows_v0' in protocol_summaries and 'direct_grouped' in protocol_summaries:
            shadows_se = protocol_summaries['classical_shadows_v0'].get('mean_se', 1)
            grouped_se = protocol_summaries['direct_grouped'].get('mean_se', 1)
            ratio = shadows_se / grouped_se if grouped_se > 0 else float('inf')
            print(f"Shadows vs Grouped SE ratio: {ratio:.2f}x")
            if ratio < 1:
                print(f"  → Classical shadows is {1/ratio:.1f}x more efficient")
            else:
                print(f"  → Direct grouped is {ratio:.1f}x more efficient")
    
    print()
    print("=" * 80)
    print(f"Full results saved to: {result.output_dir}")
    print("=" * 80)

# Generate the summary
generate_task_summary(result)

---

## Summary

The **Task Summary Report** above provides clear quantitative answers to each of the 8 Measurements Bible questions:

| Task | Answer Format |
|------|---------------|
| 1 | N* = X shots to achieve max SE ≤ ε |
| 2 | N* = X shots to achieve mean SE ≤ ε |
| 3 | Distribution statistics (mean, median, max, std) at fixed N |
| 4 | Protocol X wins on Y% of observables |
| 5 | Optimal pilot fraction = X% |
| 6 | Bias² = X, Variance = Y, MSE = Z |
| 7 | Performance degradation under noise (requires noise sweep) |
| 8 | Regret from adaptive reallocation (requires adaptive protocol) |

All results are saved to unique timestamped directories for reproducibility.